# Lesson 7: Chatting with the SEC Knowledge Graph

### Environment variables
Sets environment variables. If asked, please replace the following [your-project-id] with your project ID and run it.

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]

ACCESS_TOKEN = !gcloud auth print-access-token
ACCESS_TOKEN = ACCESS_TOKEN[0]

EMBEDDING_MODEL = "textembedding-gecko@002"

LOCATION = "us-central1"

if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [2]:
PROJECT_ID

'gpeg-oe-platform'

In [3]:
ACCESS_TOKEN

'ya29.c.c0AY_VpZjeC4T-kbE8EHa5TdBDByTAQ8-ccup9BIlRp-KHSZRkxjvwyAT5FIgyPj6OIU4rDTWQs1eeft3As6X0A_ggp8QnGqwxhLSmb1aOwRk_4VY5LGglH_hrFb0HTOSYmQiNN0_Lag909FI1tS1gGPX9Degp9tnCAIzYDkzwFEn1Nh41C3JimKzYvWYl2pucupYnnLEfbfWiXzuz54mmxPLKmrFZ1D3GBj0f9pb8337MppvCO-03uTFSBCbq3V5bvXTvZZA6FWumLRPPemOwb-vybSBcyDG96YBzI26rM-JYUSu1J10E_dhU6dZOnK2VYem-YFL_BRHSnv6rofsZHEZ6Kelk8nifIGR-4R_E6gJMdiymRXUijYOoBhGHPe2u59InL397KXsv2grz9_ud529syBYei53vpq4vOO_3nlR3wvVU4xfMa8MpxU2WB7u4Bfa55apvepZcO0vQfQei-nFOlstg52yo_hzYpnWarOS-1-xZWp1vMWZ2O6Fp2iqpO937BWMallXx-og7O8Svbk_xO10toZlMRSrJas9ZgmusY3yldIaSyZIRfQ5U9d9UFOct6wzlfuJbt5inOaO6st3y_xesJSkpj5Xonvlx4an9OclfWlX4B_kw8gSbu58ne5oBMbu_wYiVRztviBZvgwebn_odeQhJaJdtjBZ7dpB2QyUUyztwcBwVSlaMS0Z6muah6bMIMk6dXQuwgxrQJBObgXIwVW5d84UdhzRRRY5ehW3q1fJhbOxz_VuIkx7VlssUn8tz5VSYbxB5dlx7uQnZXtmY79MIVpaIJmXunspfgb9uYiOz8XwSbyUz8sllZglj-Jq0fk_asxI84XrpgrORideQVUoUZz2l-va5OvM8qfrqs47XzFutpSUXRnujs06l6B1k9BJYMevV1oqhdZ-gemYz8hdfS3XW34c4pQfpzt2dSQmja3MsJWF3tkrzFrobjbl0zlqabarirfvdOXu37hIarc

### Authentication (Colab only)
If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [4]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

<p style="background-color:#fd4a6180; padding:15px; margin-left:20px"> ⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️<b>Note:</b> This notebook takes about 30 seconds to be ready to use. Please wait until the "Kernel starting, please wait..." </p>


### Import packages and set up Neo4j

In [5]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [28]:
from dotenv import load_dotenv
import os
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

from langchain_community.llms import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [29]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [30]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

# FINAL PART

Now that everything is ready, let’s query the whole Knowledge Graph: we will provide instructions for VertexAI to respond, and also examples of Cyphers, so that LangChain generate a Cypher and give us the result based on the generated Cypher:

In [37]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain


prompt_template = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}

Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress


# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

# Give me a list of 10 companies and the value invested by them in NETAPP INC.
MATCH 
    (com:Company)-[:FILED]->(f),
    (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
RETURN mgr.managerName + " owns " + owns.shares + 
    " shares of " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) AS text
LIMIT 10

The question is:
{question}"""

cypher_prompt = PromptTemplate(
    input_variables=["schema", "question"], 
    template=prompt_template
)

cypherChain = GraphCypherQAChain.from_llm(
    VertexAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=cypher_prompt,
)

Let’s run the question and check the answer and the cypher generated:

In [38]:
cypherChain.run("How many shares of NETAPP INC does Royal Bank of Canada own?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com:Company)
WHERE mgr.managerName = "Royal Bank of Canada" AND com.companyName = "NETAPP INC"
RETURN owns.shares

Full Context:
[{'owns.shares': 842850}]

> Finished chain.


' Royal Bank of Canada owns 842850 shares of NETAPP INC.'

In [39]:
cypherChain.run("How many investors does NETAPP have?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (com:Company)-[:FILED]->(f),
    (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
WHERE com.companyName = "NETAPP INC"
RETURN count(mgr) AS investor_count;

Full Context:
[{'investor_count': 561}]

> Finished chain.


' NETAPP has 561 investors.'

In [40]:
cypherChain.run("Who is the investor with the largest NETAPP shares?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com:Company)
WHERE com.companyName = "NETAPP INC"
RETURN mgr.managerName, owns.shares
ORDER BY owns.shares DESC
LIMIT 1;

Full Context:
[{'mgr.managerName': 'VANGUARD GROUP INC', 'owns.shares': 27643006}]

> Finished chain.


' VANGUARD GROUP INC is the investor with the largest NETAPP shares.'

# THANK YOU

### Explore the updated SEC documents graph
In this lesson, you'll be working with an updated graph that also includes the address information discussed in the video
- Some outputs below may differ slightly from the video
- Start by checking the schema of the graph

In [31]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


- Check the address of a random Manager
- Note: the company returned by the following query may differ from the one in the video

In [33]:
kg.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr, addr
LIMIT 1
""")

[]

- Full text search for a manager named Royal Bank

In [34]:
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank") YIELD node, score
  RETURN node.managerName, score LIMIT 1
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 3.7019896507263184}]

- Find location of Royal Bank

In [35]:
kg.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.managerName, addr
""")

[]

- Determine which state has the most investment firms

In [18]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[]

- Determine which state has the most companies

In [19]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numCompanies
    ORDER BY numCompanies DESC
""")

[]

- What are the cities in California with the most investment firms?

In [20]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[]

- Which city in California has the most companies listed?

In [21]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

[]

- What are top investment firms in San Francisco?

In [22]:
kg.query("""
  MATCH p=(mgr:Manager)-[:LOCATED_AT]->(address:Address),
         (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
         WHERE address.city = "San Francisco"
  RETURN mgr.managerName, sum(owns.value) as totalInvestmentValue
    ORDER BY totalInvestmentValue DESC
    LIMIT 10
""")

[]

- What companies are located in Santa Clara?

In [23]:
kg.query("""
  MATCH (com:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.city = "Santa Clara"
  RETURN com.companyName
""")

[]

- What companies are near Santa Clara?

In [24]:
kg.query("""
  MATCH (sc:Address)
    WHERE sc.city = "Santa Clara"
  MATCH (com:Company)-[:LOCATED_AT]->(comAddr:Address)
    WHERE point.distance(sc.location, comAddr.location) < 10000
  RETURN com.companyName, com.companyAddress
""")

[]

- What investment firms are near Santa Clara?
- Try updating the distance in the query to expand the search radius

In [25]:
kg.query("""
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress
""")

[]

- Which investment firms are near Palo Alto Networks?
- Note that full-text search is able to handle typos!

In [26]:
# Which investment firms are near Palo Aalto Networks?
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Aalto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:LOCATED_AT]->(comAddress:Address),
    (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE point.distance(comAddress.location, 
        mgrAddress.location) < 10000
  RETURN mgr, 
    toInteger(point.distance(comAddress.location, 
        mgrAddress.location) / 1000) as distanceKm
    ORDER BY distanceKm ASC
    LIMIT 10
""")

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: fullTextCompanyNames}

- Try pausing the video and modifying queries above to further explore the graph
- You can learn more about Cypher at the neo4j website: https://neo4j.com/product/cypher-graph-query-language/ 

### Writing Cypher with an LLM

In this section, you'll use few-shot learning to teach an LLM to write Cypher
- You'll use the OpenAI's GPT 3.5 model 
- You'll also use a new Neo4j integration within LangChain called **GraphCypherQAChain**

In [27]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [ ]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [ ]:
prettyCypherChain("What investment firms are in San Francisco?")

In [ ]:
prettyCypherChain("What investment firms are in Menlo Park?")

In [ ]:
prettyCypherChain("What companies are in Santa Clara?")

In [ ]:
prettyCypherChain("What investment firms are near Santa Clara?")

### Expand the prompt to teach the LLM new Cypher patterns

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

The question is:
{question}"""

- Update Cypher generation prompt with new template, and re-initialize the Cypher chain to use the new prompt
- Rerun this code anytime you make a change to the Cypher generation template!

In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [ ]:
prettyCypherChain("What investment firms are near Santa Clara?")

### Expand the query to retrieve information from the Form 10K chunks

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)


In [ ]:
prettyCypherChain("What does Palo Alto Networks do?")

### Try for yourself!

- Update the Cypher generation prompt below to ask different questions of the graph
- You can run the "check schema" cell to be reminded of the graph structure
- Use any of the examples in this notebook, or in previous lessons, to get started
- Remember to update the prompt template and restart the GraphCypherQAChain each time you make updates!

In [ ]:
# Check the graph schema
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""

In [ ]:
# Update the prompt and reset the QA chain
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [ ]:
prettyCypherChain("<<REPLACE WITH YOUR QUESTION>>")